In [1]:
import pandas as pd
import numpy as np
import math
import operator
import os
import seaborn as sns

In [2]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.datasets import load_digits
from sklearn.metrics import pairwise_distances
from sklearn.manifold import TSNE
from sentence_transformers import SentenceTransformer
from keybert import KeyBERT
from keybert._mmr import mmr
from keybert._maxsum import max_sum_distance
from keybert._highlight import highlight_document
from keybert.backend._utils import select_backend
from tqdm import tqdm
from typing import List, Union, Tuple
from matplotlib import pyplot as plt
from scipy import linalg
from scipy.spatial.distance import squareform
from scipy.spatial.distance import pdist

/home/slava/UNSW/translationanalysis-Georgianlanguage/.venv/lib/python3.11/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
2024-08-13 22:46:21.017635: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-13 22:46:21.334523: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-13 22:46:21.460780: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin 

In [3]:
model = SentenceTransformer('all-mpnet-base-v2')

In [4]:
pd.options.display.max_colwidth = 250
texts_path = '../Georgian Texts/Vazha Pshavela/'
poems_path = texts_path + 'Poems/revised/'

In [5]:
poem_names = []
sources = ['ge/', 'en/', 'ggl/', 'gem/', 'gpt/']
ge_poems_directory = os.fsencode(poems_path + 'ge/')
for file in sorted(os.listdir(ge_poems_directory)):
    poem_names.append(file.decode())

In [6]:
print('Number of verses')
print('==============================================================================================')
print('ID\tVazha\tHewitt\tGoogle\tGemini\tChatGPT\ttPassed?\tPoem Name')
for poem in poem_names:
    output = [poem[:2]]
    for source in sources:
        with open(poems_path + source + poem, "r") as file:
            output.append(str(sum(1 for _ in file)))
    if len(set(output[1:])) > 1:
        output.append('No')
    else:
        output.append('Yes')
    output.append(poem[3:-3].replace('_', ' ').title())
    print('\t'.join(output))

Number of verses
ID	Vazha	Hewitt	Google	Gemini	ChatGPT	tPassed?	Poem Name
01	4	4	4	4	4	Yes	I Believe I Always Have Believed
02	36	36	36	36	36	Yes	Bakur
03	9	9	9	9	9	Yes	Some Day It Will Happen I Shall Die
04	14	14	14	14	14	Yes	I Feel Like Singing And I Shall Sing
05	18	18	18	18	18	Yes	Voice From The Grave
06	10	10	10	10	10	Yes	That In Truth Is Not Manliness
07	17	17	17	17	17	Yes	The Law Of The World Is Thus
08	32	32	32	32	32	Yes	Amiran
09	29	29	29	29	29	Yes	The Old Song Of Cavaliers
10	17	17	17	17	17	Yes	Consciences Song
11	12	12	12	12	12	Yes	As Once You Did O Lady As Once You Did
12	17	17	17	17	17	Yes	Loneliness
13	15	15	15	15	15	Yes	What Created Me A Human Being
14	27	27	27	27	27	Yes	Yet Again Shall I See The Spring
15	34	34	34	34	34	Yes	Thrush Its The Same Song You Sing


In [7]:
verses = []
poem_verse_counts = []
df = pd.DataFrame(
    columns=[
        'Poem ID',
        'Poem Name',
        'Verse',
        'Vazha',
        'Hewitt',
        'Google',
        'Gemini',
        'ChatGPT',
        'Hewitt - Google',
        'Hewitt - Gemini',
        'Hewitt - ChatGPT'
    ]
)

for poem in poem_names:
    poem_id = int(poem[:2])
    poem_name = poem[3:-3].replace('_', ' ').title()

    poem_vazha = [line.strip() for line in open(poems_path + 'ge/' + poem, "r")]
    poem_hewitt = [line.strip() for line in open(poems_path + 'en/' + poem, "r")]
    poem_google = [line.strip() for line in open(poems_path + 'ggl/' + poem, "r")]
    poem_gemini = [line.strip() for line in open(poems_path + 'gem/' + poem, "r")]
    poem_chatgpt = [line.strip() for line in open(poems_path + 'gpt/' + poem, "r")]

    verses.extend(poem_hewitt)
    verses.extend(poem_google)
    verses.extend(poem_gemini)
    verses.extend(poem_chatgpt)

    poem_verse_counts.append(len(poem_vazha))
    for i in range(len(poem_vazha)):
        row = {
            'Poem ID': [poem_id],
            'Poem Name': [poem_name],
            'Verse': [i + 1],
            'Vazha': [poem_vazha[i]],
            'Hewitt': [poem_hewitt[i]],
            'Google': [poem_google[i]],
            'Gemini': [poem_gemini[i]],
            'ChatGPT': [poem_chatgpt[i]]
        }
        df_new = pd.DataFrame(row)
        df = pd.concat([df, df_new], ignore_index = True)

df.shape

(291, 11)

In [8]:
df.to_csv('results/poems/all_texts.csv')

In [9]:
verse_embeddings = model.encode(verses)
verse_embeddings.shape

(1164, 768)

In [10]:
for i, row in df.iterrows():
    poem_id = row['Poem ID']
    verse = row['Verse']
    offset = sum(poem_verse_counts[:poem_id-1]) * 4
    j = offset + verse - 1

    hewitt_google = cosine_similarity(
        [verse_embeddings[j]],
        [verse_embeddings[j + 1 * poem_verse_counts[poem_id - 1]]]
    )[0][0]
    
    hewitt_gemini = cosine_similarity(
        [verse_embeddings[j]],
        [verse_embeddings[j + 2 * poem_verse_counts[poem_id - 1]]]
    )[0][0]

    hewitt_chatgpt = cosine_similarity(
        [verse_embeddings[j]],
        [verse_embeddings[j + 3 * poem_verse_counts[poem_id - 1]]]
    )[0][0]

    df.at[i,'Hewitt - Google'] = hewitt_google
    df.at[i,'Hewitt - Gemini'] = hewitt_gemini
    df.at[i,'Hewitt - ChatGPT'] = hewitt_chatgpt

In [11]:
df.head()

,Poem ID,Poem Name,Verse,Vazha,Hewitt,Google,Gemini,ChatGPT,Hewitt - Google,Hewitt - Gemini,Hewitt - ChatGPT
0,1,I Believe I Always Have Believed,1,"მრწამს, მარად მიწამებია მუდმივ სიცოცხლე სულისა, კარგისა, ქვეყნის მოყვარის, ქვეყნის ბედისგან წყლულისა.","I believe, I always have believed In the eternal life of the soul, The good, the lover of the world, Scarred by the fate of this same world.","I believe, there are always lands eternal life of the soul, Good, country lover, Ulcer from the fate of the country.","I believe, I've always believed In the eternal life of the soul, Of the good, the lover of the country, The one pained by the country's fate.","I believe, I have always believed in the eternal life of the soul, of the good, the lover of the homeland, the one suffering from the fate of the country.",0.534823,0.741352,0.767628
1,1,I Believe I Always Have Believed,2,"მრწამს, ფერფლნი კარგის გულისა ქარმ რო გაფანტოს ხმელადა, თვითოში მაინც ენთება ტიალ-სურვილი ცხელადა, –","I believe that, though the ashes of kind hearts Be scattered over the land by the wind, Yet in each of us there will burn Hotly a wish forlorn —","I believe that ashes are the heart of a good person Let the wind scatter the land, It still burns in itself Tial-desire was hot, –","I believe, even when the ashes Of a good heart are scattered by the wind, Within each one, a spark still ignites, A burning desire for warmth.","I believe, that even if the ashes of a good heart are scattered by the wind, they will still ignite a burning desire within, —",0.731435,0.758544,0.825589
2,1,I Believe I Always Have Believed,3,"ავის მჩაგრავად, კეთილის მუდამ იქნება მცველადა; ბეჩავის, გაჭირებულის მომხმარედ, მეშვლად, მხსნელადა.","To repress evil, and that this Will ever be a defence of the good, Help, succour and saviour Of the downtrodden and needy.","Evil oppressor, good There will always be a guard; Bechav, needy He helped me, helped me, saved me.","To suppress evil, to protect the good, It will always be a guardian; For the poor, the troubled, A helper, a savior, a rescuer.","as the protector against evil, the good will always be a guardian; for the helpless, the downtrodden a helper, a supporter, a savior.",0.419882,0.725808,0.590861
3,1,I Believe I Always Have Believed,4,"კარგს გულს არა ჰკლავს ბუნება, თან დააქვს ძველის-ძველადა!","Nature does not totally extinguish a kind heart, She carries it along with her from the very dawn of time.",Nature does not kill a good heart Brings old things with him!,"Nature does not kill a good heart, It carries it along, from time immemorial!","Nature does not kill a good heart, it carries the wisdom of the ancients!",0.661742,0.806899,0.761201
4,2,Bakur,1,"– გვიამბე, ელიზბარაო, რა ამბავ მოხდა ეღოსა?","Tell us, Elizbar, In Egho what did occur.","– Tell us, Elizabeth What happened to Egosa?","– Tell us, Elizbara, What happened at dawn?","Tell us, Elizbar, what happened at Egho?",0.457603,0.68623,0.907964


In [12]:
df.to_csv('results/poems/similarity/cosine_similarity.csv')

In [13]:
# maximum_difference_easwaran_gandhi.to_csv('/content/drive/MyDrive/religious_texts/semantic analysis/all-mpnet-base-v2/easwaran gandhi.csv')
# maximum_difference_gandhi_purohit.to_csv('/content/drive/MyDrive/religious_texts/semantic analysis/all-mpnet-base-v2/gandhi purohit.csv')
# maximum_difference_purohit_easwaran.to_csv('/content/drive/MyDrive/github/sentimentanalysis_bhagavadgita-main/SM_RESULTS/purohit_google.csv')
# df.to_csv('/content/drive/MyDrive/github/sentimentanalysis_bhagavadgita-main/SM_RESULTS/cosine similarity - sklearn.csv')
# df_max_sum_cs.to_csv('/content/drive/MyDrive/github/sentimentanalysis_bhagavadgita-main/SM_RESULTS/max_sum_in_every_chapter.csv')
# df_min_sum_cs.to_csv('/content/drive/MyDrive/github/sentimentanalysis_bhagavadgita-main/SM_RESULTS/least_sum_in_every_chapter.csv')
# df = pd.read_csv('/content/drive/MyDrive/github/sentimentanalysis_bhagavadgita-main/SM_RESULTS/cosine_similarity.csv')

In [14]:
maximum_difference_hewitt_google = df.loc[(df['Hewitt - Google'] < 0.5)]
maximum_difference_hewitt_google.shape

(87, 11)

In [15]:
maximum_difference_hewitt_gemini = df.loc[(df['Hewitt - Gemini'] < 0.5)]
maximum_difference_hewitt_gemini.shape

(22, 11)

In [16]:
maximum_difference_hewitt_chatgpt = df.loc[(df['Hewitt - ChatGPT'] < 0.5)]
maximum_difference_hewitt_chatgpt.shape

(19, 11)

In [17]:
maximum_difference = df.loc[(df['Hewitt - Google']<0.5) & (df['Hewitt - Gemini'] < 0.5) & (df['Hewitt - ChatGPT'] < 0.5)]
maximum_difference

,Poem ID,Poem Name,Verse,Vazha,Hewitt,Google,Gemini,ChatGPT,Hewitt - Google,Hewitt - Gemini,Hewitt - ChatGPT
7,2,Bakur,4,მე რა ვსთქვა? უნდა კაცობა ბაკურმა დაიკვეხოსა…,Who am I to speak? It is for Bak’ur To vaunt his own valour...,what can i say Must be a man Bakur boasted...,What can I say? It's for men To lament Bakuri's fate...,What should I say? It’s manhood that Bakur should boast of…,0.388787,0.320493,0.34807
12,2,Bakur,9,ბაკურის ომი და ქცევა ნეტავი თვალით გეხილა!,Bak’ur’s skill in the fight You should have seen with your own eyes!,Bakury's war and conduct I wish I could see you with my own eyes!,If only you had seen with your own eyes Bakuri's battle and fall!,Bakur’s war and actions I wish you had seen with your own eyes!,0.319068,0.415945,0.379828
16,2,Bakur,13,"– ხომ არ გაექე, რა ვუყოთ, არ დაგაცალეს სიკვდილი…",At least you didn’t flee — so why cry? They robbed you of the chance to die...,– Don't you know what to do? Death did not empty you...,"– You didn't run, what can we do, Death didn't take you...","– Maybe it wasn’t meant to be, what can we do, they didn’t let you die…",0.361001,0.459149,0.40349
84,6,That In Truth Is Not Manliness,4,"ბიჭს მაშინ დაგიძახებდი, რომ სხვის დამრჩვალსა სწამლობდე.",A man would I call you only when One throttled by another you were to tend.,I would call you a boy then To poison someone else's remains.,"I'd call you a man, If you could heal the wounds of others.","I would call you a man, If you would help the oppressed.",0.337131,0.428723,0.388801
89,6,That In Truth Is Not Manliness,9,"მგოსნობას არვის აცლიდე, მუდამ ჰკვეხდე და სცხარობდე.",When from your minstrelsy you give none peace But ever vaunt and overheat.,Don't let anyone miss you. Always boast and be proud.,"Don't deny anyone the art of poetry, Always lament and burn with passion.","To not let anyone else be a poet, To always boast and be loud.",0.166356,0.451145,0.435588
124,8,Amiran,17,"რა გაწყვეტამდის მიიყვანს ფინია რკინის მართულსო,","As soon as Pinia to breaking-point Brings the wire of iron,",How long will it take? Finnia is ruled by iron.,"Before the steed can break The iron shackles,","Whenever the dog nearly breaks the iron chain,",0.429547,0.476252,0.461807
155,9,The Old Song Of Cavaliers,16,ცოცხლები არვის დავუთმობთ მშობელი მიწის ღალასა.,"While we breathe, we concede to none The right to harvest our native soil.",We will not give up to anyone alive The parent land of Ghalasa.,We will not let anyone Betray our motherland.,We will not let anyone betray our motherland.,0.415537,0.312489,0.312489


In [18]:
df['Sum CS'] = df['Hewitt - Google'] + df['Hewitt - Gemini'] + df['Hewitt - ChatGPT']
df['Max Sum CS'] = df.groupby(['Poem Name'])['Sum CS'].transform('max')
df['Min Sum CS'] = df.groupby(['Poem Name'])['Sum CS'].transform('min')

In [19]:
df_max_sum_cs = df.loc[(df['Max Sum CS'] == df['Sum CS'])]
df_max_sum_cs.head()

,Poem ID,Poem Name,Verse,Vazha,Hewitt,Google,Gemini,ChatGPT,Hewitt - Google,Hewitt - Gemini,Hewitt - ChatGPT,Sum CS,Max Sum CS,Min Sum CS
1,1,I Believe I Always Have Believed,2,"მრწამს, ფერფლნი კარგის გულისა ქარმ რო გაფანტოს ხმელადა, თვითოში მაინც ენთება ტიალ-სურვილი ცხელადა, –","I believe that, though the ashes of kind hearts Be scattered over the land by the wind, Yet in each of us there will burn Hotly a wish forlorn —","I believe that ashes are the heart of a good person Let the wind scatter the land, It still burns in itself Tial-desire was hot, –","I believe, even when the ashes Of a good heart are scattered by the wind, Within each one, a spark still ignites, A burning desire for warmth.","I believe, that even if the ashes of a good heart are scattered by the wind, they will still ignite a burning desire within, —",0.731435,0.758544,0.825589,2.315568,2.315568,1.736551
26,2,Bakur,23,"უძილო, მშიერ-მწყურვალნი, – მტერს, რომ ჩვენ მაშინ ვიყვენით!","Without sleep, food or water — How we were then, one day the enemy will ponder! —","sleepless, hungry and thirsty, To the enemy that we were then!","Sleepless, hungry and thirsty, We were then against the enemy!","Sleepless, hungry, and thirsty, – we were to the enemy at that time!",0.763763,0.772805,0.802282,2.33885,2.33885,1.05735
42,3,Some Day It Will Happen I Shall Die,3,"ჩემს გულსა დაეხვევიან გაშმაგებულნი ჭიანი,","Frenzied worms Will wrap themselves about my heart,",They will hug my heart crazy worms,"Around my heart will coil Frenzied worms,","My heart will be encircled By frenzied worms,",0.727994,0.845164,0.891474,2.464631,2.464631,1.438483
60,4,I Feel Like Singing And I Shall Sing,12,"გაღრღნილი ჯაჭვიც მთელია, ჰხედავს, მეორე დილასა;","The gnawed-through chain is whole again, He sees it so every morning;","The worn chain is also whole, He sees it, the next morning;","The gnawed chain remains whole, He sees, on the next morning;","The gnawed chain remains whole, he sees it the next morning;",0.816771,0.870759,0.856012,2.543543,2.543543,1.458503
77,5,Voice From The Grave,15,"სული ღმერთთანა, – იქ, სადაც ბრწყინვენ ციური ძალები.","My soul is with God — there, where The heavenly powers shine.","The soul is with God, where The celestial powers shine.","My soul is with God, there, where Heavenly powers shine.","my soul is with God, – where the celestial powers shine.",0.813066,0.960912,0.926211,2.700189,2.700189,1.63788


In [20]:
df_min_sum_cs = df.loc[(df['Min Sum CS'] == df['Sum CS'])]
df_min_sum_cs.head()

,Poem ID,Poem Name,Verse,Vazha,Hewitt,Google,Gemini,ChatGPT,Hewitt - Google,Hewitt - Gemini,Hewitt - ChatGPT,Sum CS,Max Sum CS,Min Sum CS
2,1,I Believe I Always Have Believed,3,"ავის მჩაგრავად, კეთილის მუდამ იქნება მცველადა; ბეჩავის, გაჭირებულის მომხმარედ, მეშვლად, მხსნელადა.","To repress evil, and that this Will ever be a defence of the good, Help, succour and saviour Of the downtrodden and needy.","Evil oppressor, good There will always be a guard; Bechav, needy He helped me, helped me, saved me.","To suppress evil, to protect the good, It will always be a guardian; For the poor, the troubled, A helper, a savior, a rescuer.","as the protector against evil, the good will always be a guardian; for the helpless, the downtrodden a helper, a supporter, a savior.",0.419882,0.725808,0.590861,1.736551,2.315568,1.736551
7,2,Bakur,4,მე რა ვსთქვა? უნდა კაცობა ბაკურმა დაიკვეხოსა…,Who am I to speak? It is for Bak’ur To vaunt his own valour...,what can i say Must be a man Bakur boasted...,What can I say? It's for men To lament Bakuri's fate...,What should I say? It’s manhood that Bakur should boast of…,0.388787,0.320493,0.34807,1.05735,2.33885,1.05735
44,3,Some Day It Will Happen I Shall Die,5,ტყუილად აშფოთდებიან ჩემნი წაწალნი იანი.,My friends the violets In vain will be indignant.,"They worry for nothing My friends, Ian.",My swift falcons Will be troubled in vain.,My dear ones will be disturbed In vain.,0.351775,0.489707,0.597001,1.438483,2.464631,1.438483
54,4,I Feel Like Singing And I Shall Sing,6,დაჟინებულის მის ცქერით შევიქენ დასაძრახავი.,By stubborn gazing at it I have been turned into an object of reproach.,With his stubbornness I will have to move.,"With persistent gazing, I became blameworthy.",By persistently gazing at it I have become blameworthy.,0.130767,0.603262,0.724474,1.458503,2.543543,1.458503
74,5,Voice From The Grave,12,ერთხელ სჯობია სიკვდილი შავს ყოფნას ქვეყანაზედა!,A sudden death is preferable To a bleak existence on earth!,Better to die once Black presence in the country!,It's better to die once Than to live in shame on this earth!,It is better to die once than to live in darkness on earth!,0.425475,0.575076,0.637329,1.63788,2.700189,1.63788


In [21]:
print('Relation\tmean\t\t\tstd')
print(f"Hewitt-Google\t{df['Hewitt - Google'].mean()}\t{df['Hewitt - Google'].std()}")
print(f"Hewitt-Gemini\t{df['Hewitt - Gemini'].mean()}\t{df['Hewitt - Gemini'].std()}")
print(f"Hewitt-ChatGPT\t{df['Hewitt - ChatGPT'].mean()}\t{df['Hewitt - ChatGPT'].std()}")

Relation	mean			std
Hewitt-Google	0.5807760441835803	0.16068689923002036
Hewitt-Gemini	0.7074239409666291	0.1333006364700414
Hewitt-ChatGPT	0.7307519552224281	0.13619296838138836


In [22]:
df_mean_hewitt_google = df.groupby('Poem Name')['Hewitt - Google'].mean().reset_index()
df_mean_hewitt_gemini = df.groupby('Poem Name')['Hewitt - Gemini'].mean().reset_index()
df_mean_hewitt_chatgpt = df.groupby('Poem Name')['Hewitt - ChatGPT'].mean().reset_index()

df_std_hewitt_google = df.groupby('Poem Name')['Hewitt - Google'].std().reset_index()
df_std_hewitt_gemini = df.groupby('Poem Name')['Hewitt - Gemini'].std().reset_index()
df_std_hewitt_chatgpt = df.groupby('Poem Name')['Hewitt - ChatGPT'].std().reset_index()

In [23]:
print('Google', df_std_hewitt_google['Hewitt - Google'].mean())
print('Gemini', df_std_hewitt_gemini['Hewitt - Gemini'].mean())
print('ChatGPT', df_std_hewitt_chatgpt['Hewitt - ChatGPT'].mean())

Google 0.15406655603340805
Gemini 0.12116039422014982
ChatGPT 0.12653621062249046


In [24]:
kw_model = KeyBERT(model = 'all-mpnet-base-v2')

In [25]:
####### METHODOLOGY FOR OBTAINING KEYWORDS

# We encode verses using the MPNet-base model to compute the verse by verse semantic similarity. 
# We use the MPNet-base model for extraction of keywords (using KeyBERT) from all chapters. 
# However, given the constraint in the MPNet-base model that number of tokens should not exceed 384, it would not be possible to encode large chapters directly. 
# Hence, we propose a method to overcome this limitation by breaking each chapter into paragraphs of 15 verses. 
# We include 3 verses from the previous paragraph into the current paragraph to retain some context and maintain continuity. 
# For example, in the first paragraph, verses 1-15 are included, and in the second paragraph verses 13-27, then 25-39, and so on.  
# We keep the top 20 keywords because keywords that have a lower similarity score in the original paragraph may be more relevant when the entire paragraph is considered.
 
# Next, we extract the keywords for all paragraphs i with 20 candidate keywords of paragraph j such that i!=j. 
# For each keyword, we add up its cosine similarity score across paragraphs. 
# Finally, we obtain the top 10 keywords having the highest cumulative scores. 
# The key idea here is that if a term is a keyword in a certain paragraph, it also needs to be sufficiently close to other paragraphs in the higher dimensional vector space
# to qualify as a keyword for the entire chapter. 
# We use MMR with a diversity value of 0.5 to prevent the selection of similar meaning keywords.

In [28]:
top_n = 20
final_df = pd.DataFrame()

for poem in poem_names:
    df = pd.read_csv('results/poems/sentiment/' + 'en/' + poem)
    df = df.drop(['Unnamed: 0', 'Official report', 'Joking'], axis=1)

    score_list = []
    text_list = []
    string_list = []
    keyword_score_map = {}
    keyword_count_map = {}

    for i in range(math.ceil(df.shape[0] / 15)):
        if i == 0:
            text = ' '.join(df[0:15].Tweet)
        else:
            if df.shape[0] >= 15 * (i + 1) - 3:
                text = ' '.join(df[(15 * i - 3):(15 * (i + 1) - 3)].Tweet)
            else:
                text = ' '.join(df[(15 * i - 3):df.shape[0]].Tweet)

        list1 = kw_model.extract_keywords(
            text, 
            keyphrase_ngram_range=(1, 1), 
            stop_words='english', 
            top_n=top_n, 
            use_mmr=True, 
            diversity=0.5
        )
        score_list.append(list1)
        text_list.append(text)

        for jj in range(0, top_n):
            keyword = score_list[0][jj][0]
            string_list.append(keyword)
            keyword_score_map[keyword] = 0
            keyword_count_map[keyword] = 0

    for ii in range(math.ceil(df.shape[0] / 15)):
        for jj in range(math.ceil(df.shape[0] / 15)):
            list2 = kw_model.extract_keywords(
               text_list[jj], 
               candidates=string_list[top_n * ii:top_n * (ii + 1)], 
               keyphrase_ngram_range=(1, 1), 
               stop_words='english', 
               top_n=top_n,
               use_mmr=True,
               diversity=0.3
            )
            for kk in range(0, len(list2)):
                keyword = list2[kk][0]
                keyword_score = list2[kk][1]
                keyword_score_map[keyword] += keyword_score
                keyword_count_map[keyword] += 1


    keyword_score_map = dict( sorted(keyword_score_map.items(), key=operator.itemgetter(1), reverse=True))   # sort in descending order according to scores
    keyword_df = pd.DataFrame(keyword_score_map.items(), columns=['Keyword', 'Score'])                       # convert dictionary to dataframe
    keyword_df = keyword_df[:5]                                                                              # TOP 10 KEYWORDS most relevant to text 
    keyword_df['Poem'] = poem[3:-3].replace('_', ' ').title()
    keyword_df['Author'] = 'Hewitt'

    final_df = pd.concat([final_df, keyword_df], axis=0)

final_df.to_csv('results/poems/keywords/en/keywords.csv')

In [29]:
df_keywords_en = pd.read_csv('results/poems/keywords/en/keywords.csv')
df_keywords_ggl = pd.read_csv('results/poems/keywords/ggl/keywords.csv')
df_keywords_gem = pd.read_csv('results/poems/keywords/gem/keywords.csv')
df_keywords_gpt = pd.read_csv('results/poems/keywords/gpt/keywords.csv')

df_keywords_en = df_keywords_en.drop(['Unnamed: 0', 'Score', 'Author'], axis=1)
df_keywords_en = df_keywords_en.reindex(columns=['Poem', 'Keyword'])
df_keywords_en = pd.concat([
        df_keywords_en,
        df_keywords_ggl['Keyword'],
        df_keywords_gem['Keyword'],
        df_keywords_gpt['Keyword']
    ],     
    axis=1
)
df_keywords_en.columns = ['Poem', 'Hewitt', 'Google', 'Gemini', 'ChatGPT']
df_keywords_en.to_csv('results/poems/keywords/all.csv')

In [30]:
keyword_dict = dict()
for i, row in df_keywords_en.iterrows():
    poem = row['Poem']
    if poem not in keyword_dict:
        keyword_dict[poem] = {"Hewitt": set(), "Google": set(), "Gemini": set(), "ChatGPT": set()}
    keyword_dict[poem]["Hewitt"].add(row["Hewitt"])
    keyword_dict[poem]["Google"].add(row["Google"])
    keyword_dict[poem]["Gemini"].add(row["Gemini"])
    keyword_dict[poem]["ChatGPT"].add(row["ChatGPT"])

keyword_count = dict()
for poem in keyword_dict.keys():
    keyword_count[poem] = {"Google": 0, "Gemini": 0, "ChatGPT": 0}
    keyword_count[poem]["Google"] = len(keyword_dict[poem]["Hewitt"].intersection(keyword_dict[poem]["Google"]))
    keyword_count[poem]["Gemini"] = len(keyword_dict[poem]["Hewitt"].intersection(keyword_dict[poem]["Gemini"]))
    keyword_count[poem]["ChatGPT"] = len(keyword_dict[poem]["Hewitt"].intersection(keyword_dict[poem]["ChatGPT"]))

model_count = {"Google": 0, "Gemini": 0, "ChatGPT": 0}
for _, model in keyword_count.items():
    model_count["Google"] += model["Google"]
    model_count["Gemini"] += model["Gemini"]
    model_count["ChatGPT"] += model["ChatGPT"]

model_count

{'Google': 15, 'Gemini': 27, 'ChatGPT': 22}

In [31]:
round(22 / 75 * 100, 2)

29.33

In [35]:
df = pd.read_csv('results/poems/keywords/gpt/keywords.csv')
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.drop_duplicates(subset='Keyword', keep='last', inplace=True)
onlykey = df.loc[(df['Author'] == 'ChatGPT')]
onlykey = onlykey['Keyword']
onlykey.to_csv('results/poems/keywords/gpt/onlykey.tsv', sep='\t', index=None, header=None)

In [36]:
keyword_score_map = dict(sorted(keyword_score_map.items(), key=operator.itemgetter(1), reverse=True))
df = pd.DataFrame(keyword_score_map.items(), columns=['Keyword', 'Score'])
df

,Keyword,Score
0,heart,2.3169
1,god,1.7871
2,mind,1.3680
3,yearning,1.3443
4,mourn,1.2516
5,tree,0.9531
6,song,0.8436
7,troubled,0.7977
8,years,0.7749
9,sweetly,0.7476


In [37]:
model = KeyBERT(model='all-mpnet-base-v2')

In [38]:
df = pd.read_csv('results/poems/keywords/en/keywords.csv')
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.drop_duplicates(subset='Keyword', keep='last', inplace=True)
df

,Keyword,Score,Poem,Author
1,eternal,0.3774,I Believe I Always Have Believed,Hewitt
2,evil,0.3584,I Believe I Always Have Believed,Hewitt
3,kind,0.3397,I Believe I Always Have Believed,Hewitt
4,fate,0.3353,I Believe I Always Have Believed,Hewitt
5,bak,1.7298,Bakur,Hewitt
...,...,...,...,...
70,heart,2.3169,Thrush Its The Same Song You Sing,Hewitt
71,god,1.7871,Thrush Its The Same Song You Sing,Hewitt
72,mind,1.3680,Thrush Its The Same Song You Sing,Hewitt
73,yearning,1.3443,Thrush Its The Same Song You Sing,Hewitt


In [39]:
candidates = df['Keyword'].tolist()
doc,candidate_embeddings = model.extract_embeddings(candidates)
print(len(candidate_embeddings))
print(len(candidate_embeddings[0]))

63
768


In [40]:
candidate_embeddings

array([[-0.0107369 ,  0.01089811,  0.00692198, ...,  0.06413829,
        -0.05025541, -0.01605672],
       [ 0.0189002 , -0.09099139, -0.00339644, ...,  0.0348289 ,
        -0.01164863, -0.0428898 ],
       [-0.03342865,  0.04567591, -0.0067057 , ..., -0.01460963,
         0.0023025 , -0.00601836],
       ...,
       [-0.00249044,  0.02519077, -0.00256387, ...,  0.03261773,
         0.02467911, -0.03140422],
       [-0.02500432,  0.08243218, -0.00419225, ...,  0.04901766,
         0.00605663, -0.00770791],
       [-0.00172552,  0.03948552,  0.00199088, ...,  0.01448915,
         0.00865195,  0.01055847]], dtype=float32)

In [41]:
candidate_embeddings = pd.DataFrame(candidate_embeddings)
candidate_embeddings

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
0,-0.010737,0.010898,0.006922,-0.005416,0.023905,-0.030482,0.040101,0.042587,-0.046637,0.007654,...,-0.020223,0.008975,0.100679,0.033649,-0.003748,0.031828,-0.005312,0.064138,-0.050255,-0.016057
1,0.018900,-0.090991,-0.003396,0.006611,-0.045337,0.000062,0.026904,-0.016521,0.013034,-0.004391,...,0.007916,-0.013134,0.111735,-0.044299,0.026516,0.142247,0.002976,0.034829,-0.011649,-0.042890
2,-0.033429,0.045676,-0.006706,0.000623,0.005098,-0.003458,-0.006733,-0.005518,-0.051397,-0.017533,...,-0.054720,-0.009401,0.007813,0.027395,-0.023441,-0.039752,-0.006506,-0.014610,0.002303,-0.006018
3,-0.035719,-0.032315,-0.033645,0.003220,-0.020609,0.041257,-0.007759,0.011344,-0.004202,-0.031080,...,-0.003944,0.029458,0.037410,-0.035367,0.012898,0.003860,0.020805,0.050928,-0.080401,-0.001199
4,0.065701,0.089237,-0.016188,-0.000267,-0.002430,0.018057,-0.025891,0.037742,-0.023782,0.000353,...,-0.003463,0.057465,0.045809,0.022666,0.011349,0.063853,0.020731,0.027494,-0.004350,-0.026027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58,0.035809,0.040539,-0.022638,-0.004772,-0.000918,0.030015,-0.038228,0.002170,-0.031875,-0.024016,...,-0.017242,0.041852,0.034591,-0.011919,0.006209,0.000045,-0.020167,0.009215,-0.061911,0.006222
59,0.051936,-0.014338,-0.012240,-0.023119,0.001501,-0.016430,-0.082047,0.032840,-0.036672,-0.012158,...,-0.025608,-0.030747,0.056350,-0.010051,-0.026033,-0.017203,-0.016116,0.025665,0.010380,-0.016304
60,-0.002490,0.025191,-0.002564,0.018941,-0.007700,0.004254,-0.047994,0.002343,-0.061513,0.035401,...,-0.027401,0.007419,0.040168,0.033805,0.016771,0.049601,0.000799,0.032618,0.024679,-0.031404
61,-0.025004,0.082432,-0.004192,0.012045,-0.067833,0.017840,-0.001997,0.007650,0.060157,0.003402,...,0.015074,-0.009519,0.049978,0.005677,-0.021754,-0.044486,-0.020297,0.049018,0.006057,-0.007708


In [42]:
candidate_embeddings.to_csv('results/poems/keywords/en/all.tsv', sep='\t', index=None, header=None)
df.to_csv('results/poems/keywords/en/all.tsv', index=False, sep='\t')

In [43]:
sns.set(rc={'figure.figsize':(11.7,8.27)})
palette = sns.color_palette("bright", 2)
tsne = TSNE(perplexity=30)
X_embedded = tsne.fit_transform(candidate_embeddings)

sns.scatterplot(x=X_embedded[:,0], y=X_embedded[:,1], legend='full')
plt.xlabel(xlabel='T-SNE dimension 1')
plt.ylabel(ylabel='T-SNE dimension 2')
plt.show()